In [43]:
import numpy as np
import os
import matmul as mm
import math_util as mu
import scipy.io as io
from amm_methods import *
import socket # Obtain the current host name, which can be used to select different data directories and result saving directories

In [44]:
# method = METHOD_MITHRAL
method = METHOD_PQ
# method = METHOD_EXACT
# method = METHOD_SCALAR_QUANTIZE

In [45]:
feedback_bits = 256
ncodebooks=128
ncentroids=256
train_sam_num = 3000 # 训练集样本数
split = 16 # A的列、B的行分割成split份

In [ ]:
# zx = np.random.randint(1,100,(11,3))
# print(zx)
# mins_new, ranges_new = saturation_sort(zx)
# print(mins_new, ranges_new)

In [46]:
host_name = socket.gethostname()
if host_name == 'DESKTOP-PLRL7TK':
    dir_train = 'E:\\hdr\\研一\\华为-深度学习\\intermediate\\intermediate8dbfc1'
    dir_result = ''
elif host_name == 'DESKTOP-6FOH47P':
    dir_train = 'F:\\Projects\\python\\PQ\\intermediate8dbfc1'
    dir_result = 'F:\\Projects\\python\\PQ\\res'
    data_to_fcpath_train= 'data_to_fc_e39_7999.npy'
    featurepath_train= 'feature_e39_7999.npy'
    data_to_fcpath_test = 'data_to_fc_e39_7999.npy'
    featurepath_test = 'feature_e39_7999.npy'
elif host_name == 'jm-System-Product-Name':
    dir_train = '/data/hdr/transformer_data/joined'
    dir_result = '/data/hdr/pq/res'
    data_to_fcpath_train= 'data_to_fc_train_f%i_sam%i.npy' % (feedback_bits, train_sam_num)
    y_train = 'y_train_f%i_sam%i.npy' % (feedback_bits, train_sam_num)
    featurepath_train= 'feature_train_f%i_sam%i.npy' % (feedback_bits, train_sam_num)
    data_to_fcpath_test = 'data_to_fc_test.npy'
    featurepath_test = 'feature_test.npy'
    y_test = 'y_test.npy'
else:
    raise NameError("You are running the script in a new computer, please define dir_train")

dir_train_split = os.path.join(dir_train, 'train', 'f'+str(feedback_bits), 'split'+str(split))
try:
    os.mkdir(dir_train_split)
except FileExistsError:
    pass
# print(dir_train_split)
dir_test_split = os.path.join(dir_train, 'test', 'f'+str(feedback_bits), 'split'+str(split))
try:
    os.mkdir(dir_test_split)
except FileExistsError:
    pass 


weightpath = 'encoder_fcw.npy'
biaspath = 'encoder_fcb.npy'
dir_result = os.path.join(dir_result, method)


切分训练集

In [47]:
data_to_fc_train = np.load(os.path.join(dir_train, data_to_fcpath_train))
print(data_to_fc_train.shape)
split_size = int(data_to_fc_train.shape[1] / split) # split后单个训练集A的列数
data_to_fc_train_split_path_list = []
for i in range(split):
    data_to_fc_train_split_path_list.append(os.path.join(dir_train_split, 'data_to_fc_train_f'+str(feedback_bits)+'_split'+str(split)+'_'+str(i)+'.npy'))
    np.save(data_to_fc_train_split_path_list[i], data_to_fc_train[np.ix_(range(data_to_fc_train.shape[0]), range(i*split_size,(i+1)*split_size))])

(96000, 2048)


In [48]:
weight_train = np.load(os.path.join(dir_train, weightpath))
print(weight_train.shape)
weight_train_split_path_list = []
for i in range(split):
    weight_train_split_path_list.append(os.path.join(dir_train_split, 'weight_train_f'+str(feedback_bits)+'_split'+str(split)+'_'+str(i)+'.npy'))
    np.save(weight_train_split_path_list[i], weight_train[np.ix_(range(i*split_size,(i+1)*split_size), range(weight_train.shape[1]))])

(2048, 256)


In [49]:
# 输入与weight相乘
y_train_split_path_list = []
for i in range(split):
    y_train_split_path_list.append(os.path.join(dir_train_split, 'y_train_f'+str(feedback_bits)+'_split'+str(split)+'_'+str(i)+'.npy'))
    xx = np.load(data_to_fc_train_split_path_list[i])
    ww = np.load(weight_train_split_path_list[i])
    np.save(y_train_split_path_list[i], np.matmul(xx, ww))

切分测试集

In [50]:
data_to_fc_test = np.load(os.path.join(dir_train, data_to_fcpath_test))
print(data_to_fc_test.shape)
# split_size = int(data_to_fc_train.shape[1] / split) # split后单个训练集A的列数
data_to_fc_test_split_path_list = []
for i in range(split):
    data_to_fc_test_split_path_list.append(os.path.join(dir_test_split, 'data_to_fc_test_f'+str(feedback_bits)+'_split'+str(split)+'_'+str(i)+'.npy'))
    np.save(data_to_fc_test_split_path_list[i], data_to_fc_test[np.ix_(range(data_to_fc_test.shape[0]), range(i*split_size,(i+1)*split_size))])

(32000, 2048)


In [51]:
# 输入与weight相乘
y_test_split_path_list = []
for i in range(split):
    y_test_split_path_list.append(os.path.join(dir_test_split, 'y_test_f'+str(feedback_bits)+'_split'+str(split)+'_'+str(i)+'.npy'))
    xx = np.load(data_to_fc_test_split_path_list[i])
    ww = np.load(weight_train_split_path_list[i])
    np.save(y_test_split_path_list[i], np.matmul(xx, ww))

In [52]:
est_list = []
for i in range(split):
    dir_est, X_path = os.path.split(data_to_fc_train_split_path_list[i])
    dir_est, W_path = os.path.split(weight_train_split_path_list[i])
    dir_est, Y_path = os.path.split(y_train_split_path_list[i])
    est3 = mm.estFactory(X_path=X_path, W_path=W_path, Y_path=Y_path, dir= dir_est, ncodebooks=ncodebooks, ncentroids=ncentroids, methods=[method])
    est_list.append(est3)


running method:  PQ
running kmeans in subspace 1/128... mse / {var(X_subs), var(X)}: 0.000276, 7.12e-05
running kmeans in subspace 2/128... mse / {var(X_subs), var(X)}: 0.000175, 9.04e-06
running kmeans in subspace 3/128... mse / {var(X_subs), var(X)}: 0.000762, 8.64e-05
running kmeans in subspace 4/128... mse / {var(X_subs), var(X)}: 0.000134, 7.87e-06
running kmeans in subspace 5/128... mse / {var(X_subs), var(X)}: 0.00107, 9.83e-05
running kmeans in subspace 6/128... mse / {var(X_subs), var(X)}: 0.00185, 6.31e-05
running kmeans in subspace 7/128... mse / {var(X_subs), var(X)}: 0.000955, 8.48e-05
running kmeans in subspace 8/128... mse / {var(X_subs), var(X)}: 0.00397, 0.00136
running kmeans in subspace 9/128... mse / {var(X_subs), var(X)}: 0.0021, 0.00177
running kmeans in subspace 10/128... mse / {var(X_subs), var(X)}: 0.00113, 0.00231
running kmeans in subspace 11/128... mse / {var(X_subs), var(X)}: 0.000383, 0.000902
running kmeans in subspace 12/128... mse / {var(X_subs), var(X)

In [53]:
y_split_out_matmul_list = []
for i in range(split):
    x_test = np.load(data_to_fc_test_split_path_list[i])
    w_test = np.load(weight_train_split_path_list[i])
    y_split_out_matmul = mm.eval_matmul(est_list[i], x_test, w_test)
    y_split_out_matmul_list.append(y_split_out_matmul)

In [54]:
# split 的矩阵结果合成
y_out_matmul = y_split_out_matmul_list[0]
if split>1:
    for i in range(1, split):
        y_out_matmul += y_split_out_matmul_list[i]

In [55]:
bias = np.load(dir_train+'/'+biaspath)
y_out_last = y_out_matmul + bias.T # MADDNESS替换后当前层输出，即+bias并不需要激活函数后的结果

In [56]:
print(y_out_last)
print(y_out_last.shape)
# np.save("LDPC_decoder_NET_testdata/" + snr + "nomul_matmul_yout_matmul", y_out_matmul)
# np.save(dir_result+'/'+method+'fc1_fb256_cb%i_ct%i.npy' % (ncodebooks, ncentroids), y_out_matmul)
np.save(dir_result+'/'+method+'fc1_split%i_fb%i_cb%i_ct%i.npy' % (split, feedback_bits, ncodebooks, ncentroids), y_out_last)
# io.savemat(dir_result+'\\fc1_256.mat', {"NN_output_buffer": y_out_last})

[[-0.54656    -0.42330346 -0.9887754  ... -0.50818586 -0.33817065
  -0.44142818]
 [-0.66756386 -0.5639416  -0.72495246 ... -0.8240569  -0.41092712
  -0.5213005 ]
 [-0.07835776 -0.48236194 -0.6632153  ... -0.91431296 -0.24543951
  -0.2955555 ]
 ...
 [-0.08613387 -0.29495054 -0.9313994  ... -0.60089445 -0.63593626
  -0.4424353 ]
 [-0.18556586 -0.43336508 -0.6779109  ... -0.3748304  -0.43990916
  -0.38616365]
 [-0.7377552  -0.39339876 -0.86364067 ... -0.77854955 -0.5752109
  -0.31930545]]
(32000, 256)
